# Part 1

In [1]:
import numpy as np
import pandas as pd
from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error as mae

In [2]:
def get_sample(df,n):
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

In [3]:
def imae(x,y): return (abs(x-y)).mean()

In [4]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

In [5]:
def print_score(m):
    maer = [imae(m.predict(X_train), y_train), imae(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): maer.append(m.oob_score_)
    print(maer)

In [6]:
def numericalize(df, col, name, max_n_cat):
    if not  pd.api.types.is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

In [7]:
def fix_missing(df, col, name, na_dict):
    if  pd.api.types.is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

In [8]:
def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not pd.api.types.is_numeric_dtype(df[y_fld]): df[y_fld] = pd.Categorical(df[y_fld]).codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [9]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

# Part 2

In [10]:
TRAIN_FILE='/home/thiago.martinelli/code/main/data/pubg/train_V2.csv'
TEST_FILE='/home/thiago.martinelli/code/main/data/pubg/test_V2.csv'

In [12]:
df_raw = pd.read_csv(TRAIN_FILE, engine='python')

In [13]:
train_cats(df_raw)

In [14]:
df1 = df_raw.set_index('matchId',drop=False)

In [15]:
dfr=df_raw

In [16]:
df3 = df1.copy()

In [17]:
ind = df1.index.value_counts()
dind = dict(ind)

In [18]:
df2 = df1.sort_index()
df3 = df2.iloc[:55500,:]
matches=dict(df3.index.value_counts())

In [45]:
df29=df3.copy()

In [46]:
for match in set(df3.index):
    if match not in m:
        print(match)
        df3.drop(match,axis=0,inplace=True)

01cc8f9b15d3d6


/home/thiago.martinelli/venvs/p3/lib/python3.5/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


02cf1db07c3934
0195fbe135716e
001e7bc06b1611
0287a3001f1447
02f656fa780115
02ca9ace246f02
031c7b39718669
03044c944dde28
034a385dd060d4
0314fb2fb89340
02954a1ce681c0
02db72db1ecd2d
005aca992c9a78
011b0592429c85
02dba39585334b
013bead1c1736c
007edf38197cec
01ce13ad81c6f2
003c2b333747d9
0104eeb664494d
02d17b99f39a48
01ca563b808a48
009505123eaceb
011f1afe2db83d
030ce58f65df30
0101ab001bb2c2
00b217e69ba569
029e7db020600e
01aead02bb8901
02b4a6e639a609
01ee716430b23a
00930789f7075f
017563d39f54b4
01794e4f556206
01370c58128112
02fa6b06351447
02e566e5b8618f
00c0586a397e12
00f2dad24ae3a1
030eff2bfd4ad1
019e9035fb8c79
016d03d56d3dc1
0211af1f386a89
00a4afdb911815
01530800ed9120
020b0802aba439
01ad9ab6e931b8
02ceab5f102b39
0165d05e93e628
014f0eac673783
015c7dc3bfecf6
03445f606ff7f2
00f600051f84ac
01435d33376fd0
00df303460dfe6
016c4b1866bece
0249eb2549b198
001cd8e7e6b737
0027b1ffb2e346
00bf40fb9aa15c
023e7d5684fe09
01223b3a01f9dc
029b8d960efd17


In [47]:
def rank(a):
    c=list(a)
    b = np.asarray(list(set(np.sort(c))))
    for n,i in enumerate(a):
        a[n] = np.where(b==i)[0]
    return a
  
    

In [48]:
def norma(a):
    for n,i in enumerate(a):
        a[n] = float(a[n] / a.max())
    return a

In [49]:
def feat_eng(df,cols):
    newdf = pd.DataFrame()
    %time
    count=0
    for i, ndf in df.groupby(level=0):
        count+=1
        print(count, end='\r')
        for c in cols:
            serie=ndf[c]
            rank(np.asarray(serie))
            ndf[c + 'Pos'] = serie
        newdf = pd.concat( (newdf, ndf), axis=0)
    return newdf

In [50]:
def feat_eng2(df,cols):
    newdf = pd.DataFrame()
    %time
    count=0
    for mi in set(df['matchId']):
        ndf = df[df['matchId']==mi]
        count+=1
        print(count, end='\r')
        for c in cols:
            serie=ndf[c]
            serie=rank(np.asarray(serie))
            #serie=norma(serie)
            ndf[c + 'Pos'] = serie
        newdf = pd.concat( (newdf, ndf), axis=0)
    return newdf

In [56]:
def feat_eng3(df,cols):
    newdf = pd.DataFrame()
    %time
    count=0
    #for mi in set(df.index):
    for c in cols:
        df[c]=df.groupby('matchId').apply(rank)
    return newdf

In [52]:
%time dfa, _, _ = proc_df(df3)
mt = dfa.copy()

CPU times: user 212 ms, sys: 20 ms, total: 232 ms
Wall time: 239 ms


In [53]:
c1=list(['killStreaks'])

In [54]:
mt=mt.drop('matchId',axis=1)

In [57]:
df6=feat_eng3(mt,c1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 21.9 µs


ValueError: Length of values does not match length of index

In [101]:
cols = list(['assists', 'boosts', 'damageDealt', 'DBNOs', \
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', \
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace', \
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills', \
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', \
       'weaponsAcquired', 'winPoints'])


In [120]:
c2=list(['headshotKills','maxPlace'])

In [122]:
mt2=mt.iloc[:1000,:]

In [123]:
newdf = feat_eng2(mt2,c2)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.6 µs


/home/thiago.martinelli/venvs/p3/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [124]:
display_all(newdf.head(100).T)

matchId,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000a43bce5eec,0000eb01ea6cdd,0000eb01ea6cdd,0000eb01ea6cdd,0000eb01ea6cdd,0000eb01ea6cdd
Id,4.443029e+06,3.319870e+05,293764.000,415831.000,3.860556e+06,1068895.000,1.616288e+06,2.449703e+06,1.257083e+06,2.665784e+06,1.742702e+06,617164.000,4.335817e+06,4.308171e+06,2.957133e+06,421552.000,3.642291e+06,4384175.000,1909818.000,3.571300e+06,2.067946e+06,4.043874e+06,1.602834e+06,4.264765e+06,1.209189e+06,9.069290e+05,1.741871e+06,2.931100e+05,2133090.0,7.996360e+05,9.147500e+04,3.996233e+06,2.017131e+06,3.168556e+06,2.624957e+06,3.549925e+06,2.302140e+05,2.437963e+06,2.005776e+06,3.502335e+06,3.653795e+06,3.443301e+06,3964037.000,35174.0000,38839.0000,4.264347e+06,1538612.000,2.886511e+06,1567446.000,2739580.000,9.233060e+05,3.124751e+06,1.393298e+06,4024426.000,2.427663e+06,2.092852e+06,1.982436e+06,1.500499e+06,9.012050e+05,9.134730e+05,3.624573e+06,4.781510e+05,2.882225e+06,2.318280e+06,2.798432e+06,3.675110e+06,1.419566e+06,6.859790e+05,1.422029e+06,1.872709e+06,2.562149e+06,3.901800e+05,2.479088e+06,2.925295e+06,3.556463e+06,2.124464e+06,3.801216e+06,4.040552e+06,1.594177e+06,3.738702e+06,2585121.000,3.722148e+06,728674.0000,1.716296e+06,2.992680e+06,2.471510e+06,5.964480e+05,1.202292e+06,1.789834e+06,4.416604e+06,1.924356e+06,670964.0000,3.799990e+06,4.396726e+06,1.999491e+06,2.090921e+06,674968.0000,4.203934e+06,1.019291e+06,3.392996e+06
groupId,7.115560e+05,1.199250e+06,612351.000,280836.000,1.244160e+06,280836.000,1.427215e+06,7.115560e+05,9.820900e+05,1.199250e+06,8.593150e+05,1599340.000,1.095869e+06,1.147078e+06,1.844836e+06,280836.000,1.839290e+06,463843.000,1599340.000,1.147078e+06,1.527915e+06,1.641338e+06,1.916779e+06,1.147078e+06,7.115560e+05,1.131961e+06,1.131961e+06,1.803227e+06,463843.0,1.244160e+06,1.244160e+06,1.595120e+06,1.310968e+06,8.106500e+05,1.746278e+06,1.527915e+06,1.147078e+06,9.399720e+05,1.427215e+06,1.147078e+06,1.095869e+06,1.467244e+06,280836.000,982090.0000,195603.0000,1.803227e+06,1599340.000,8.106500e+05,280836.000,612351.000,1.199250e+06,1.310968e+06,1.199250e+06,280836.000,9.399720e+05,8.106500e+05,1.746278e+06,1.095869e+06,1.516323e+06,1.641338e+06,1.839290e+06,1.844836e+06,9.399720e+05,1.746278e+06,9.820900e+05,1.244160e+06,8.593150e+05,1.244160e+06,1.839290e+06,1.839290e+06,8.106500e+05,1.844836e+06,1.467244e+06,9.820900e+05,1.527915e+06,9.820900e+05,1.199250e+06,6.030640e+05,1.803227e+06,1.147078e+06,1599340.000,1.427215e+06,603064.0000,1.199250e+06,1.310968e+06,1.641338e+06,1.844836e+06,9.399720e+05,1.956030e+05,1.916779e+06,1.916779e+06,859315.0000,1.527915e+06,1.916779e+06,1.131961e+06,2.3

In [126]:
df_tr = newdf.drop(['winPlacePerc'],axis=1)
y_tr = newdf['winPlacePerc']

n_valid = 100  # same as Kaggle's test set size
n_tr = len(df_tr)-n_valid
#raw_train, raw_valid = split_vals(df_raw, n_tr)
X_train, X_valid = split_vals(df_tr, n_tr)
y_train, y_valid = split_vals(y_tr, n_tr)
X_train.shape, y_train.shape, X_valid.shape

m = RandomForestRegressor(n_jobs=-1, n_estimators = 50 )
m.fit(X_train, y_train)
print_score(m)

[0.02175403460670194, 0.05691404314285713, 0.9868445674673793, 0.9112186780436199]
